Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [2]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-01-10 16:28:53--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.225.107
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.225.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’

instacart_online_gr 100%[===================>] 196.03M  45.8MB/s    in 4.7s    

2020-01-10 16:28:58 (41.8 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’ saved [205548478/205548478]



In [3]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [4]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [5]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
##### YOUR CODE HERE #####

import pandas as pd

In [8]:
order_products__prior = pd.read_csv('order_products__prior.csv')

order_products__prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [9]:
order_products__train = pd.read_csv('order_products__train.csv')

order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [10]:
orders = pd.read_csv('orders.csv')

orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [11]:
products = pd.read_csv('products.csv')

products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [0]:
top_product_names = ['Banana', 
                     'Bag of Organic Bananas',
                     'Organic Strawberries',
                     'Organic Baby Spinach',
                     'Organic Hass Avocado',
                     'Organic Avocado',
                     'Large Lemon',
                     'Strawberries',
                     'Limes',
                     'Organic Whole Milk']

In [0]:
top_products = products[products['product_name'].isin(top_product_names)]

top_products

,product_id,product_name,aisle_id,department_id
13175,13176,Bag of Organic Bananas,24,4
16796,16797,Strawberries,24,4
21136,21137,Organic Strawberries,24,4
21902,21903,Organic Baby Spinach,123,4
24851,24852,Banana,24,4
26208,26209,Limes,24,4
27844,27845,Organic Whole Milk,84,16
47208,47209,Organic Hass Avocado,24,4
47625,47626,Large Lemon,24,4
47765,47766,Organic Avocado,24,4


In [0]:
top_products = top_products.drop(['aisle_id', 'department_id'], axis=1)

top_products

,product_id,product_name
13175,13176,Bag of Organic Bananas
16796,16797,Strawberries
21136,21137,Organic Strawberries
21902,21903,Organic Baby Spinach
24851,24852,Banana
26208,26209,Limes
27844,27845,Organic Whole Milk
47208,47209,Organic Hass Avocado
47625,47626,Large Lemon
47765,47766,Organic Avocado


In [0]:
product_names = products.drop(['aisle_id', 'department_id'], axis=1)

product_names.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [0]:
total_orders = pd.concat([order_products__prior, order_products__train])

total_orders.shape

total_orders.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [0]:
orders_with_product_names = pd.merge(total_orders, product_names, on='product_id', how='inner')

orders_with_product_names.head()

,order_id,product_id,add_to_cart_order,reordered,product_name
0,2,33120,1,1,Organic Egg Whites
1,26,33120,5,0,Organic Egg Whites
2,120,33120,13,0,Organic Egg Whites
3,327,33120,5,1,Organic Egg Whites
4,390,33120,28,1,Organic Egg Whites


In [0]:
orders_with_product_names['product_name'].value_counts()[:10]

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [0]:
# table2 -> tidy
# Put treatments column into the table
table2 = table2.reset_index()
table2

,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [0]:
tidy2 = table2.melt(id_vars='index', value_vars=['John Smith', 'Jane Doe', 'Mary Johnson'])

tidy2

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [0]:
tidy2 = tidy2.rename(columns={
    'index': 'trt',
    'variable': 'name',
    'value': 'result'
})

tidy2

,trt,name,result
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [0]:
# tidy -> table2

wide = tidy2.pivot_table(index='trt', columns='name', values='result')

wide

name,Jane Doe,John Smith,Mary Johnson
trt,,,
treatmenta,16.0,NaN,3.0
treatmentb,11.0,2.0,1.0


In [0]:
import seaborn as sns

flights = sns.load_dataset('flights')

flights.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [0]:
# Flights Pivot Table

wide = flights.pivot_table(index='year', columns='month', values='passengers')

wide

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [0]:
orders_per_product = orders_with_product_names['product_name'].value_counts().rename_axis('name').reset_index(name='counts')
print(orders_per_product.shape)
orders_per_product.head()

(49685, 2)


,name,counts
0,Banana,491291
1,Bag of Organic Bananas,394930
2,Organic Strawberries,275577
3,Organic Baby Spinach,251705
4,Organic Hass Avocado,220877


In [0]:
popular_products = orders_per_product[orders_per_product['counts'] >= 2900]
print(popular_products.shape)
popular_products.head()

(2053, 2)


,name,counts
0,Banana,491291
1,Bag of Organic Bananas,394930
2,Organic Strawberries,275577
3,Organic Baby Spinach,251705
4,Organic Hass Avocado,220877


In [0]:
# Now I need to get the latest ordered

clean_orders = orders.drop(['user_id', 'eval_set', 'order_number', 'order_dow', 'days_since_prior_order'], axis=1)

orders_with_hours = pd.merge(orders_with_product_names, clean_orders, on='order_id')

orders_with_hours.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,order_hour_of_day
0,2,33120,1,1,Organic Egg Whites,9
1,2,28985,2,1,Michigan Organic Kale,9
2,2,9327,3,0,Garlic Powder,9
3,2,45918,4,1,Coconut Butter,9
4,2,30035,5,0,Natural Sweetener,9


In [0]:
orders_with_hours_clean = orders_with_hours.drop(['add_to_cart_order', 'reordered'], axis=1)
print(orders_with_hours_clean.shape)
orders_with_hours_clean.head()

(33819106, 4)


,order_id,product_id,product_name,order_hour_of_day
0,2,33120,Organic Egg Whites,9
1,2,28985,Michigan Organic Kale,9
2,2,9327,Garlic Powder,9
3,2,45918,Coconut Butter,9
4,2,30035,Natural Sweetener,9


In [0]:
products_with_mean_hour = orders_with_hours_clean.groupby('product_name', as_index=False)['order_hour_of_day'].mean()

In [0]:
products_with_mean_hour = products_with_mean_hour.rename(columns={
    'product_name': 'name'
})

products_with_mean_hour.head()

,name,order_hour_of_day
0,#2 Coffee Filters,13.330413
1,#2 Cone White Coffee Filters,12.000000
2,#2 Mechanical Pencils,12.000000
3,#4 Natural Brown Coffee Filters,12.607287
4,& Go! Hazelnut Spread + Pretzel Sticks,13.229885


In [0]:
merged = pd.merge(popular_products, products_with_mean_hour, on='name')

merged.head()

,name,counts,order_hour_of_day
0,Banana,491291,13.393433
1,Bag of Organic Bananas,394930,13.376434
2,Organic Strawberries,275577,13.467223
3,Organic Baby Spinach,251705,13.465795
4,Organic Hass Avocado,220877,13.482585


In [0]:
merged.sort_values(by='order_hour_of_day', ascending=False)[:25]

,name,counts,order_hour_of_day
1985,Half Baked Frozen Yogurt,2983,14.526986
1891,The Tonight Dough™ Ice Cream,3137,14.526299
878,Americone Dream® Ice Cream,6447,14.394912
914,Half Baked® Ice Cream,6222,14.388942
1870,Phish Food® Ice Cream,3177,14.373308
1093,Milk and Cookies Ice Cream,5223,14.369711
1887,Sicilian Pistachio,3146,14.306421
1296,Chocolate Peanut Butter Cup Gelato,4511,14.303480
1037,Birthday Cake Light Ice Cream,5462,14.292018
1300,Cherry Garcia Ice Cream,4495,14.270300


## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [12]:
##### YOUR CODE HERE #####

order_products__prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [13]:
order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [14]:
order_products = pd.concat([order_products__prior, order_products__train])
print(order_products.shape)
order_products.head()

(33819106, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [15]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [16]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [17]:
aisles = pd.read_csv('aisles.csv')

aisles.head(25)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
5,6,other
6,7,packaged meat
7,8,bakery desserts
8,9,pasta sauce
9,10,kitchen supplies


In [18]:
product_names = products.drop(['product_name', 'department_id'], axis=1)

product_names.head()

,product_id,aisle_id
0,1,61
1,2,104
2,3,94
3,4,38
4,5,5


In [19]:
product_aisles = pd.merge(product_names, aisles)

product_aisles.head()

,product_id,aisle_id,aisle
0,1,61,cookies cakes
1,78,61,cookies cakes
2,102,61,cookies cakes
3,172,61,cookies cakes
4,285,61,cookies cakes


In [20]:
# Order products slimmed down

order_products = order_products.drop(['add_to_cart_order'], axis=1)

order_products.head()

,order_id,product_id,reordered
0,2,33120,1
1,2,28985,1
2,2,9327,0
3,2,45918,1
4,2,30035,0


In [21]:
order_products_with_aisles = pd.merge(order_products, product_aisles, on='product_id')
print(order_products_with_aisles.shape)
order_products_with_aisles.head()

(33819106, 5)


,order_id,product_id,reordered,aisle_id,aisle
0,2,33120,1,86,eggs
1,26,33120,0,86,eggs
2,120,33120,0,86,eggs
3,327,33120,1,86,eggs
4,390,33120,1,86,eggs


In [22]:
# Now I need to get total orders per product
total_orders_per_aisle = order_products_with_aisles['aisle'].value_counts().rename_axis('aisle').reset_index(name='total_orders')
total_orders_per_aisle.head()

,aisle,total_orders
0,fresh fruits,3792661
1,fresh vegetables,3568630
2,packaged vegetables fruits,1843806
3,yogurt,1507583
4,packaged cheese,1021462


In [24]:
# Get total reorders per product
total_reorders_per_aisle = order_products_with_aisles.groupby('aisle', as_index=False)['reordered'].sum()

total_reorders_per_aisle.head()

,aisle,reordered
0,air fresheners candles,6500
1,asian foods,60986
2,baby accessories,4669
3,baby bath body care,2626
4,baby food formula,233042


In [25]:
# Merge both dataframes

merged = pd.merge(total_orders_per_aisle, total_reorders_per_aisle)

merged.head()

,aisle,total_orders,reordered
0,fresh fruits,3792661,2726251
1,fresh vegetables,3568630,2123540
2,packaged vegetables fruits,1843806,1178700
3,yogurt,1507583,1034957
4,packaged cheese,1021462,598280


In [27]:
# Create % reordered column

merged['percent_reorder'] = merged['reordered'] / merged['total_orders']

merged.head()

,aisle,total_orders,reordered,percent_reorder
0,fresh fruits,3792661,2726251,0.718823
1,fresh vegetables,3568630,2123540,0.595057
2,packaged vegetables fruits,1843806,1178700,0.639275
3,yogurt,1507583,1034957,0.686501
4,packaged cheese,1021462,598280,0.585710
